In [ ]:
import requests
import zipfile
import io
from os import path

temp_dir = "temp"

resources_url = "https://zenodo.org/record/5520883/files/resources.zip?download=1"
resources_dest_dir = path.dirname(path.realpath("__file__"))

# # Get resources from Zento
response = requests.get(resources_url)
zipped_data = zipfile.ZipFile(io.BytesIO(response.content))
zipped_data.extractall(resources_dest_dir)


In [ ]:

# Get Geonames data
geonames_url = "http://download.geonames.org/export/dump/GB.zip"
geonames_dest_dir = path.join(resources_dest_dir, "resources/geonames/")

response = requests.get(geonames_url)
zipped_data = zipfile.ZipFile(io.BytesIO(response.content))
zipped_data.extractall(geonames_dest_dir)

alt_geonames_url = "http://download.geonames.org/export/dump/alternateNamesV2.zip"
response = requests.get(alt_geonames_url)
zipped_data = zipfile.ZipFile(io.BytesIO(response.content))
zipped_data.extractall(geonames_dest_dir)

In [ ]:


# Get Boundary-Line™ Data
boundaryline_url = "https://osdatahub.os.uk/downloads/open/BoundaryLine"
boundaryline_url = "https://api.os.uk/downloads/v1/products/BoundaryLine/downloads?area=GB&format=ESRI%C2%AE+Shapefile&redirect"
boundaryline_dest_dir = path.join(resources_dest_dir, "resources/geoshapefiles/")


response = requests.get(boundaryline_url)
zipped_data = zipfile.ZipFile(io.BytesIO(response.content))
zipped_data.extractall(boundaryline_dest_dir)


In [ ]:
ranklib_url = "https://sourceforge.net/projects/lemur/files/lemur/RankLib-2.13/RankLib-2.13.jar/download"
rank_lib_fpath = path.join(resources_dest_dir, "resources/ranklib/RankLib-2.13.jar")

response = requests.get(ranklib_url)
with open(rank_lib_fpath, mode='wb') as jar_file:
    jar_file.write(response.content)

In [ ]:
# Wikidata
wikidata_url = "https://dumps.wikimedia.org/wikidatawiki/entities/latest-all.json.bz2"
wikidata_fpath = path.join(resources_dest_dir, "resources/wikidata/latest-all.json.bz2")

chunk_size = 1024*1024
i = 0

#  session = requests.Session()
response = requests.get(wikidata_url, stream=True)
print(response.status_code)
print(response.ok)
with open(wikidata_fpath, mode="wb") as fb:
    for chunk in response.iter_content(chunk_size=chunk_size):
        print(f"{i*chunk_size}")
        fb.write(chunk)
        i = i+1



In [36]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import re

def get_details_from_nginx_filelist(filelist_url, filename):
    search_fname = re.escape(filename)
    pattern = (r"^(?P<filename>" +
                search_fname +
                r")\s+(?P<datetime>\d\d-[a-zA-Z]{3}-20\d\d \d\d:\d\d)\s+(?P<filesize>\d+)"
    )

    response = requests.get(filelist_url)

    # As a minimum, check that it is an nginx site
    if not re.search("nginx", response.headers["server"]):
        raise ValueError(f"Not an Nginx webserver: {filelist_url}")

    file_list_txt = BeautifulSoup(response.text).get_text()
    match = re.search(pattern, file_list_txt, re.MULTILINE)

    if match:
        return match.group("filename"), match.group("datetime"), int(match.group("filesize"))

    return None
    # filesize
    print(file_list_txt)


wikidata_filelist_url = "https://dumps.wikimedia.org/wikidatawiki/entities"
wikidata_fname = "latest-all.json.bz2"

_, _, target_size = get_details_from_nginx_filelist(wikidata_filelist_url, wikidata_fname)
print(target_size)
print(path.getsize(wikidata_fpath))

if path.getsize(wikidata_fpath) != target_size:
    print("warning - wikidata not downloaded correctly")


73207061006
17350737578
warning - wikidata not downloaded correctly
